# Convert skymap files

## Overview

In this notebook, we will provide brief overviews of:
- what skymaps are
- why we need a way to convert their structure/representation
- how they're currently stuctured in lsst.skymaps
- a couple different strategies we use for this conversion

### What is a skymap?

(a brief explanation)

(maybe also describe how lsst structures/stores skymaps?)

### Why do we care?

- lsdb now enables users to search by tract/patch, as they would on the RSP
- however, using the lsst skymap requries the lsst skymap package, which depends on other packages in the lsst stack
- this means it isn't pip installable :(
- so, we provide infrastrucutre to read in these skymap files and convert them to new formats that can be read without the lsst stack

## Methods of converting and storing skymaps

There are a couple of methods we explore:

### "Full Vertex"
- see code/detailed description in notebook (name)
- this is a vertex dump
- we calculate the vertices for each tract in the skymap file, then dump into the yaml as radec values
- really has no deps which is nice
- and works for more or less any kind of skymap
- however, storage isn't ideal
- (metrics)
- plus, if we added patches, at the 10x10 patches per tract that lsst uses,
- (metrics)
- so, we use prior knowledge of the lsst's ring structure to 

### "Ring Optimized"
- see code/detailed description in notebook (name)
- `ringSkymap` files, of which the lsst skymap is a type, divide the sky into a set of horizontal "rings"
- they have consistent upper, lower dec boundaries
- they are subdivided into tracts, which (besides the poles) are all rectangular (rounding for floating point precision; conservatively beyond arcseconds)
- (could list some more invariants about rings and tracts)
- (metrics)

## Other information

There are a few more things to know about skymaps.

- inner vs outer rings
- nobody really knows the boundaries?
- maybe other things, idk